In [ ]:

# If you run this notebook separately, uncomment this line
#%run AnalysisFunctions.ipynb


In [ ]:
#filtered = ""

In [ ]:
print("Running Dede Analysis")


print(f"Apply Filter?: {apply_filter}")

if apply_filter.lower() == "true":
    filtered = "/Filtered"
    print("Compiling results with filtered pairs")
else:
    filtered = ""
    print("Compiling results with unfiltered pairs")

In [ ]:
# zdLFC does not filter out anything so filtered = unfiltered
zdLFC = pd.read_csv("../zdLFC Scripts/zdLFC Output/DeDe_zdLFC.csv")

zdLFC = zdLFC.rename(columns={zdLFC.columns[0]: 'GENE'})

zdLFC = zdLFC.set_index('GENE')
zdLFC
zdLFC = reindex_alphbetically(zdLFC)
zdLFC = -1 * zdLFC
zdLFC


In [ ]:
orthrus = pd.read_csv("../Orthrus Scripts/OrthrusOutput" + filtered + "/dede_orthrus.tsv", sep='\t')
orthrus['GENE'] = orthrus['gene1'] + "_" + orthrus['gene2']

orthrus =  orthrus[['GENE','differential_combn_vs_single_A549', 'differential_combn_vs_single_HT29',  'differential_combn_vs_single_OVCAR8']]
orthrus = orthrus.rename(columns={'differential_combn_vs_single_A549': 'A549', 'differential_combn_vs_single_HT29': 'HT29', 'differential_combn_vs_single_OVCAR8': 'OVCAR8'})
orthrus = orthrus.set_index('GENE')
orthrus = reindex_alphbetically(orthrus)
orthrus = orthrus * -1
orthrus


In [ ]:
gemini_sens = pd.read_csv("../Gemini Scripts/GeminiOutput" + filtered + "/Gemini_Dede_Sensitive_Lethality.csv")
gemini_sens = gemini_sens.rename(columns={gemini_sens.columns[0]: 'gene_pair'})
gemini_sens['gene_pair'] = gemini_sens['gene_pair'].str.replace(';', '_')
gemini_sens = gemini_sens.set_index('gene_pair')
gemini_sens = reindex_alphbetically(gemini_sens)


In [ ]:
gemini_strong = pd.read_csv("../Gemini Scripts/GeminiOutput" + filtered + "/Gemini_Dede_Strong.csv")
gemini_strong = gemini_strong.rename(columns={gemini_strong.columns[0]: 'gene_pair'})
gemini_strong['gene_pair'] = gemini_strong['gene_pair'].str.replace(';', '_')
gemini_strong = gemini_strong.set_index('gene_pair')
gemini_strong = reindex_alphbetically(gemini_strong)


In [ ]:
# load the latest parrish 
# Parrish also does not filter out any pairs so didnt run it by remnoving filtering steps
parrish = pd.read_csv('../Parrish Score Scripts/ParrishOutput/Dede_Parrish.csv')
parrish.set_index('paralog_pair', inplace = True)

parrish = reindex_alphbetically(parrish)
parrish = parrish * -1
parrish

In [ ]:
Dede_A549 = pd.concat([zdLFC['A549'], orthrus['A549'],  gemini_strong['A549'],gemini_sens['A549'], parrish['GI_score_A549']], axis = 1,
                     keys= scores)

results_Dede_A549 = Cell_Line_Analysis(Dede_A549, 'A549', 'Dede', scores, ground_truth_depmap_hit)



In [ ]:
# thankfull not major changes in aupr and auroc
Dede_HT29 = pd.concat([zdLFC['HT29'], orthrus['HT29'], gemini_strong['HT29'], gemini_sens['HT29'], parrish['GI_score_HT29']], axis = 1,
                     keys=scores)

results_Dede_HT29 = Cell_Line_Analysis(Dede_HT29, 'HT29', 'Dede', scores, ground_truth_depmap_hit)



In [ ]:
Dede_OVCAR8 = pd.concat([zdLFC['OVCAR8'], orthrus['OVCAR8'], gemini_strong['OVCAR8'], gemini_sens['OVCAR8'], parrish['GI_score_OVCAR8']], axis = 1,
                     keys=scores)

results_Dede_OVCAR8 = Cell_Line_Analysis(Dede_OVCAR8, 'OVCAR8', 'Dede', scores,ground_truth_depmap_hit)



In [ ]:
full_list = [results_Dede_A549, results_Dede_HT29, results_Dede_OVCAR8]
roc, pr = ConvertResultsToDF(full_list)

In [ ]:
## Write results to a file
roc.to_csv("Output" + filtered + "/Dede_ROC_DepMapHits.csv", index=False)
pr.to_csv("Output" + filtered + "/Dede_PR_DepMapHits.csv", index=False)


In [ ]:
# combine all cell lines and then repeat analysis
dede_all = pd.concat([Dede_A549, Dede_OVCAR8,Dede_HT29])
dede_all
results_Dede_All = Cell_Line_Analysis(dede_all, 'All', 'Dede', scores, ground_truth_depmap_hit)


roc_all, pr_all = ConvertResultsToDF([results_Dede_All] )


In [ ]:
dede_all.corr(method = "spearman").to_csv("Output" + filtered + "/Dede_Correlation.csv")

In [ ]:
## Write results to a file
roc_all.to_csv("Output" + filtered + "/Dede_All_ROC_DepMapHits.csv", index=False)
pr_all.to_csv("Output" + filtered + "/Dede_All_PR_DepMapHits.csv", index=False)


In [ ]:
 ##### NOW USE GROUND TRUTH KORLEFE 1  #######

results_Dede_A549_korlefe =Cell_Line_Analysis(Dede_A549, 'A549', 'Dede', scores, Koferle)
results_Dede_HT29_korlefe =Cell_Line_Analysis(Dede_HT29, 'HT29', 'Dede', scores, Koferle)
results_Dede_OVCAR8_korlefe =Cell_Line_Analysis(Dede_OVCAR8, 'OVCAR8', 'Dede', scores, Koferle)

full_list_Koferle = [results_Dede_A549_korlefe, results_Dede_HT29_korlefe,results_Dede_OVCAR8_korlefe ]
roc_Koferle, pr_Koferle = ConvertResultsToDF(full_list_Koferle )


roc_Koferle.to_csv("Output" + filtered + "/Dede_ROC_Koferle.csv", index=False)
pr_Koferle.to_csv("Output" + filtered + "/Dede_PR_Koferle.csv", index=False)


results_Dede_all_Koferle = Cell_Line_Analysis(dede_all, 'All', 'Dede', scores, Koferle)

roc_all_Koferle, pr_all_Koferle = ConvertResultsToDF([results_Dede_all_Koferle] )

roc_all_Koferle.to_csv("Output" + filtered + "/Dede_All_ROC_Koferle.csv", index=False)
pr_all_Koferle.to_csv("Output" + filtered + "/Dede_All_PR_Koferle.csv", index=False)


In [ ]:
## Jaccard Similarity for gene pairs
Dede_OVCAR8_ = jaccard_similarity(Dede_OVCAR8, 0.95)
Dede_A549_ = jaccard_similarity(Dede_A549, 0.95)
Dede_HT29_ = jaccard_similarity(Dede_HT29, 0.95)

In [ ]:
Dede_OVCAR8_.to_csv("Output" + filtered + "\Comparison\Dede_OVCAR8_Jaccard.csv")
Dede_A549_.to_csv("Output" + filtered + "\Comparison\Dede_A549_Jaccard.csv")
Dede_HT29_.to_csv("Output" + filtered + "\Comparison\Dede_HT29_Jaccard.csv")
Dede_Jaccard = (Dede_OVCAR8_ + Dede_A549_ +Dede_HT29_ ) / 3
Dede_Jaccard.to_csv("Output" + filtered + "\Comparison\Dede_Jaccard.csv")

In [ ]:
dede_all.corr(method = "spearman").to_csv("Output" + filtered + "\Comparison\Dede_Correlations.csv")